# Technical Analysis Indicator Price Prediction
The goal of this project is to analyze the predictive power of the top 10 most popular TA indicators and see how well they do to predict price over a 30 day period. I am going to find the value of the indicators on day 1 (30 trading days ago) and then find the daily closing price for 30 days later and measure how well the indicator predicted the price.  

first we'll find the top 500 stocks by market cap from nasdaq and pull them into a dataframe


In [1]:
# Importing pandas library for data manipulation and analysis
import pandas as pd

# Load the CSV file into a DataFrame
csv_file_path = '/Users/evancallaghan/Downloads/nasdaq_screener_1726538993372.csv' 
df = pd.read_csv(csv_file_path)

# Inspect the DataFrame to understand its structure
print(df.head())

# Filter DataFrame to only show the columns 'Symbol', 'Name', and 'Market Cap'
df = df[['Symbol', 'Name', 'Market Cap']]

# Convert 'Market Cap' to numeric if it's not already
# Remove commas, dollar signs, and replace these symbols with empty spaces
df['Market Cap'] = df['Market Cap'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Sort the DataFrame by Market Cap in descending order
df_sorted = df.sort_values(by='Market Cap', ascending=False).head(5000)                                                                        
df_sorted.head()


  Symbol                                               Name Last Sale  \
0      A             Agilent Technologies Inc. Common Stock   $138.31   
1     AA                    Alcoa Corporation Common Stock     $34.50   
2   AACG   ATA Creativity Global American Depositary Shares   $0.5025   
3   AACT  Ares Acquisition Corporation II Class A Ordina...    $10.80   
4   AADI                  Aadi Bioscience Inc. Common Stock     $1.88   

   Net Change % Change    Market Cap        Country  IPO Year    Volume  \
0      1.0000   0.728%  3.974029e+10  United States    1999.0    887040   
1      1.9800   6.089%  8.912735e+09  United States    2016.0  10730428   
2     -0.0275  -5.189%  1.608006e+07          China    2008.0     25043   
3      0.0200   0.186%  0.000000e+00            NaN    2023.0     35074   
4      0.0800   4.444%  4.627589e+07  United States       NaN     81942   

        Sector                                          Industry  
0  Industrials  Biotechnology: Laboratory A

,Symbol,Name,Market Cap
15,AAPL,Apple Inc. Common Stock,3.288959e+12
4208,MSFT,Microsoft Corporation Common Stock,3.206167e+12
4559,NVDA,NVIDIA Corporation Common Stock,2.864613e+12
2819,GOOG,Alphabet Inc. Class C Capital Stock,1.957167e+12
2820,GOOGL,Alphabet Inc. Class A Common Stock,1.945719e+12


In [2]:
# Reset the index of the DataFrame and drop the old index
df_sorted.reset_index(drop=True, inplace=True)

# Update the index to start from 1 instead of 0
df_sorted.index = df_sorted.index + 1

# Display the first few rows of the updated DataFrame
df_sorted.head()

,Symbol,Name,Market Cap
1,AAPL,Apple Inc. Common Stock,3.288959e+12
2,MSFT,Microsoft Corporation Common Stock,3.206167e+12
3,NVDA,NVIDIA Corporation Common Stock,2.864613e+12
4,GOOG,Alphabet Inc. Class C Capital Stock,1.957167e+12
5,GOOGL,Alphabet Inc. Class A Common Stock,1.945719e+12


remove all stocks except common stocks

In [3]:
# Ensure there are no leading or trailing whitespaces in the 'Name' column
df_sorted['Name'] = df_sorted['Name'].str.strip()

# List of terms to filter out
terms_to_drop = ["Capital Stock", "Depository Shares", "Global Notes", "ADS", 
                 "Registry Shares", "Depositary Shares"
]

# Create a regex pattern to match any of the terms
# //b ensures that the match occues only at the start or end of a word
# pipe '|' ensures that if any of the terms in 'terms_to_drop' are seen, 
# there is a match
pattern = '|'.join([f"\\b{term}\\b" for term in terms_to_drop])

# Apply filtering based on the updated pattern
df_filtered = df_sorted[~df_sorted['Name'].str.contains(pattern, case=False, 
                                                        na=False)
]

# Display the filtered DataFrame
df_filtered.head()

,Symbol,Name,Market Cap
1,AAPL,Apple Inc. Common Stock,3.288959e+12
2,MSFT,Microsoft Corporation Common Stock,3.206167e+12
3,NVDA,NVIDIA Corporation Common Stock,2.864613e+12
5,GOOGL,Alphabet Inc. Class A Common Stock,1.945719e+12
6,AMZN,Amazon.com Inc. Common Stock,1.940525e+12


In [7]:
# Reset the index of the DataFrame and drop the old index
df_filtered.reset_index(drop=True, inplace=True)

# Update the index to start from 1 instead of 0
df_filtered.index = df_filtered.index + 1

# Display the first few rows of the updated DataFrame
df_filtered.head()

,Symbol,Name,Market Cap
1,AAPL,Apple Inc. Common Stock,3.288959e+12
2,MSFT,Microsoft Corporation Common Stock,3.206167e+12
3,NVDA,NVIDIA Corporation Common Stock,2.864613e+12
4,GOOGL,Alphabet Inc. Class A Common Stock,1.945719e+12
5,AMZN,Amazon.com Inc. Common Stock,1.940525e+12


below are the 10 technical indicators we are going to use for this project.
1. Relative Strength Index (RSI)
2. Moving Average Convergence Divergence (MACD)
3. Stochastic Oscillator
4. Simple Moving Average (SMA)
5. Exponential Moving Average (EMA)
6. Volume Weighted Average Price (VWAP)
7. Bollinger Bands
8. Average True Range (ATR)
9. Fibonacci Retracement 

In [ ]:
# There is a CSV file I have provided so this code does not need to be run again
# Computationally intensive

import yfinance as yf
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Function to download stock data for a single stock
def download_stock_data(ticker):
    try:
        data = yf.download(ticker, start="2022-02-10", end="2025-02-10", interval="1d")[['Close', 'High', 'Low', 'Volume']]
        if data.empty:
            print(f"Warning: No data found for {ticker} (possibly due to non-trading days like weekends or holidays)")
            return None  # Return None if the data is empty
        return data
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
        return None  # Return None if there is any error (e.g., stock not found)

# List of tickers from your df_filtered dataframe
tickers = df_filtered['Symbol'].tolist()

# Batch size for processing tickers in chunks
batch_size = 100

# Create a function to download data for a batch of tickers in parallel
def download_batch(batch_tickers):
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(download_stock_data, batch_tickers))
    # Remove None values from the results
    return [result for result in results if result is not None]

# Loop through the tickers in batches
for i in range(0, len(tickers), batch_size):
    batch_tickers = tickers[i:i + batch_size]
    results = download_batch(batch_tickers)

    if results:  # Check if results are not empty
        # Combine all individual stock data into a single dataframe
        df_batch = pd.concat(results, keys=batch_tickers)

        # Save the data to CSV for the current batch
        df_batch.to_csv(f'/content/drive/MyDrive/stock_data_yahoo_{i // batch_size}.csv')
        print(f"Downloaded batch {i // batch_size} and saved to CSV")
    else:
        print(f"Batch {i // batch_size} has no data. Skipping...")
        # Optionally, log the tickers that failed for this batch
        print(f"Failed tickers in batch {i // batch_size}: {batch_tickers}")


In [10]:
import pandas as pd

# Initialize an empty list to hold DataFrames
df_list = []

# List of specific file indices
file_indices = [0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 27, 28, 29, 30, 45, 44, 26, 18, 46]

# Loop through the specific CSV file indices
for i in file_indices:
    # Construct the file path for each batch
    csv_file_path = f'/Users/evancallaghan/flatiron_ds/phase_5/capstone_project/stock_data_yahoo_{i}.csv'

    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Rename 'Price' column to 'Symbol'
    df = df.rename(columns={'Price': 'Symbol'})

    # Append the DataFrame to the list
    df_list.append(df)

# Concatenate all DataFrames in the list along the rows (axis=0)
df_all = pd.concat(df_list, ignore_index=True)

# Display the first few rows of the merged DataFrame
df_all.head()


/var/folders/wv/49phbq0x3bj2l3xqln6zr3x40000gn/T/ipykernel_90942/562544689.py:15: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
/var/folders/wv/49phbq0x3bj2l3xqln6zr3x40000gn/T/ipykernel_90942/562544689.py:15: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
/var/folders/wv/49phbq0x3bj2l3xqln6zr3x40000gn/T/ipykernel_90942/5625446

,Symbol,Unnamed: 1,Close,Close.1,High,High.1,Low,Low.1,Volume,Volume.1,...,Low.21,Volume.21,Close.22,High.22,Low.22,Volume.22,Close.23,High.23,Low.23,Volume.23
0,Ticker,NaN,CCCS,POST,CCCS,POST,CCCS,POST,CCCS,POST,...,WTS,WTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAPL,2022-02-10,10.520000457763672,69.12957763671875,10.729999542236328,70.2225112915039,10.199999809265137,68.92015838623047,1037700.0,642524.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,2022-02-11,10.09000015258789,69.76439666748047,10.489999771118164,70.5235595703125,10.020000457763672,68.95942687988281,480300.0,492169.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAPL,2022-02-14,10.220000267028809,71.02094268798828,10.460000038146973,71.27617645263672,9.970000267028809,69.64659881591797,724400.0,672473.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Get a list of distinct tickers in the 'Symbol' column
distinct_symbols = df_all['Symbol'].unique()

# Count the number of distinct symbols
num_distinct_symbols = len(distinct_symbols)

# Print the result
print(f"Number of distinct symbols: {num_distinct_symbols}")

# Optionally, print the list of distinct symbols (first 10 for brevity)
print(f"List of distinct symbols (first 10): {distinct_symbols[:10]}")


Number of distinct symbols: 1792
List of distinct symbols (first 10): ['Ticker' nan 'AAPL' 'MSFT' 'NVDA' 'GOOGL' 'AMZN' 'META' 'BRK/A' 'BRK/B']


In [12]:
# Get the list of unique symbols from df_all and df_filtered
unique_symbols_all = df_all['Symbol'].unique()
unique_symbols_filtered = df_filtered['Symbol'].unique()

# Find the symbols that are in df_all but not in df_filtered, ensuring all items are strings
symbols_not_in_filtered = [str(symbol) for symbol in unique_symbols_all if str(symbol) not in map(str, unique_symbols_filtered)]

# Print the list of symbols that are not in df_filtered
print('\n'.join(symbols_not_in_filtered))


Ticker
nan


In [13]:
import numpy as np

# Convert 'nan' strings or any NaN-like values to actual np.nan
df_all['Symbol'] = df_all['Symbol'].apply(lambda x: np.nan if (isinstance(x, str) and x.lower() == 'nan') or pd.isna(x) else x)

# Drop rows where the 'Symbol' column contains NaN values
df_all_cleaned = df_all.dropna(subset=['Symbol'])

# Verify the result by checking the first few rows
df_all_cleaned.head()


,Symbol,Unnamed: 1,Close,Close.1,High,High.1,Low,Low.1,Volume,Volume.1,...,Low.21,Volume.21,Close.22,High.22,Low.22,Volume.22,Close.23,High.23,Low.23,Volume.23
0,Ticker,NaN,CCCS,POST,CCCS,POST,CCCS,POST,CCCS,POST,...,WTS,WTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAPL,2022-02-10,10.520000457763672,69.12957763671875,10.729999542236328,70.2225112915039,10.199999809265137,68.92015838623047,1037700.0,642524.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,2022-02-11,10.09000015258789,69.76439666748047,10.489999771118164,70.5235595703125,10.020000457763672,68.95942687988281,480300.0,492169.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAPL,2022-02-14,10.220000267028809,71.02094268798828,10.460000038146973,71.27617645263672,9.970000267028809,69.64659881591797,724400.0,672473.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AAPL,2022-02-15,10.5600004196167,71.5575942993164,10.569999694824219,72.25785064697266,10.220000267028809,70.99476623535156,758700.0,380319.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# This will ensure that if any main column (Close, High, Low, or Volume)
# has missing values, they will be filled with the corresponding values
# from the suffixed columns (e.g., Close.1, Close.2, etc.), and once done,
# the suffixed columns will be removed from the DataFrame.

import re

# List of main columns
main_columns = ['Close', 'High', 'Low', 'Volume']

# Iterate over the main columns to check and replace NaN values with corresponding suffixed columns
for col in main_columns:
    # Look for columns with numeric suffixes like .1, .2, .3, ..., .23
    suffix_columns = [col + '.' + str(i) for i in range(1, 24)]  # Create the list of possible suffixes

    # For each suffix column, if it exists, fill NaN in the main column with its values
    for suffix_col in suffix_columns:
        if suffix_col in df_all.columns:
            df_all[col] = df_all[col].fillna(df_all[suffix_col])

    # After filling NaN values from the suffixed columns, drop the suffixed columns
    df_all_cleaned = df_all.drop(columns=suffix_columns)

# Verify the changes
print(df_all_cleaned.head())


   Symbol  Unnamed: 1               Close            Close.1  \
0  Ticker         NaN                CCCS               POST   
1     NaN        Date                 NaN                NaN   
2    AAPL  2022-02-10  10.520000457763672  69.12957763671875   
3    AAPL  2022-02-11   10.09000015258789  69.76439666748047   
4    AAPL  2022-02-14  10.220000267028809  71.02094268798828   

                 High             High.1                 Low  \
0                CCCS               POST                CCCS   
1                 NaN                NaN                 NaN   
2  10.729999542236328   70.2225112915039  10.199999809265137   
3  10.489999771118164   70.5235595703125  10.020000457763672   
4  10.460000038146973  71.27617645263672   9.970000267028809   

               Low.1     Volume Close.2  ... Low.20 Close.21 High.21 Low.21  \
0               POST       CCCS    LOAR  ...    KBH      WTS     WTS    WTS   
1                NaN        NaN     NaN  ...    NaN      NaN     NaN    

In [15]:
df_all_cleaned.columns

Index(['Symbol', 'Unnamed: 1', 'Close', 'Close.1', 'High', 'High.1', 'Low',
       'Low.1', 'Volume', 'Close.2', 'High.2', 'Low.2', 'Close.3', 'High.3',
       'Low.3', 'Close.4', 'High.4', 'Low.4', 'Close.5', 'High.5', 'Low.5',
       'Close.6', 'High.6', 'Low.6', 'Close.7', 'High.7', 'Low.7', 'Close.8',
       'High.8', 'Low.8', 'Close.9', 'High.9', 'Low.9', 'Close.10', 'High.10',
       'Low.10', 'Close.11', 'High.11', 'Low.11', 'Close.12', 'High.12',
       'Low.12', 'Close.13', 'High.13', 'Low.13', 'Close.14', 'High.14',
       'Low.14', 'Close.15', 'High.15', 'Low.15', 'Close.16', 'High.16',
       'Low.16', 'Close.17', 'High.17', 'Low.17', 'Close.18', 'High.18',
       'Low.18', 'Close.19', 'High.19', 'Low.19', 'Close.20', 'High.20',
       'Low.20', 'Close.21', 'High.21', 'Low.21', 'Close.22', 'High.22',
       'Low.22', 'Close.23', 'High.23', 'Low.23'],
      dtype='object')

In [16]:
# List of columns you want to keep
columns_to_keep = ['Symbol', 'Unnamed: 1', 'Close', 'High', 'Low', 'Volume']

# Select only the columns you want to keep and drop the others
df_all_cleaned = df_all_cleaned[columns_to_keep]

# Verify the result by checking the first few rows
df_all_cleaned.head()


,Symbol,Unnamed: 1,Close,High,Low,Volume
0,Ticker,NaN,CCCS,CCCS,CCCS,CCCS
1,NaN,Date,NaN,NaN,NaN,NaN
2,AAPL,2022-02-10,10.520000457763672,10.729999542236328,10.199999809265137,1037700.0
3,AAPL,2022-02-11,10.09000015258789,10.489999771118164,10.020000457763672,480300.0
4,AAPL,2022-02-14,10.220000267028809,10.460000038146973,9.970000267028809,724400.0


In [17]:
# Rename the 'Unnamed: 1' column to 'Date'
df_all_cleaned = df_all_cleaned.rename(columns={'Unnamed: 1': 'Date'})

# Verify the change by checking the first few rows
df_all_cleaned.head()


,Symbol,Date,Close,High,Low,Volume
0,Ticker,NaN,CCCS,CCCS,CCCS,CCCS
1,NaN,Date,NaN,NaN,NaN,NaN
2,AAPL,2022-02-10,10.520000457763672,10.729999542236328,10.199999809265137,1037700.0
3,AAPL,2022-02-11,10.09000015258789,10.489999771118164,10.020000457763672,480300.0
4,AAPL,2022-02-14,10.220000267028809,10.460000038146973,9.970000267028809,724400.0


In [18]:
# Drop rows with index 0 and 1
df_all_cleaned = df_all_cleaned.drop([0, 1])

# Reset the index
df_all_cleaned = df_all_cleaned.reset_index(drop=True)

# Verify the changes by checking the first few rows
df_all_cleaned.head()


,Symbol,Date,Close,High,Low,Volume
0,AAPL,2022-02-10,10.520000457763672,10.729999542236328,10.199999809265137,1037700.0
1,AAPL,2022-02-11,10.09000015258789,10.489999771118164,10.020000457763672,480300.0
2,AAPL,2022-02-14,10.220000267028809,10.460000038146973,9.970000267028809,724400.0
3,AAPL,2022-02-15,10.5600004196167,10.569999694824219,10.220000267028809,758700.0
4,AAPL,2022-02-16,10.4399995803833,10.550000190734863,10.390000343322754,685200.0


In [20]:
# Check for non-numeric values in the 'Close' column
non_numeric_values = df_all_cleaned[~df_all_cleaned['Close'].apply(pd.to_numeric, errors='coerce').notna()]
print(non_numeric_values[['Date', 'Symbol', 'Close']].head())

        Date  Symbol Close
57472    NaN  Ticker  MASI
57473   Date     NaN   NaN
109293   NaN  Ticker  BILL
109294  Date     NaN   NaN
169643   NaN  Ticker   NVO
